In [50]:
!pip install pandas numpy matplotlib seaborn
!pip install pulp
!pip install glpk
import pandas as pd
import numpy as np
import pulp
import time
from datetime import datetime
import matplotlib.pyplot as plt



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [51]:
# 模型1:目标函数只考虑了“最大化教室利用率”
import pandas as pd
import numpy as np

def optimize_course_scheduling(courses_df, rooms_df):
    # 预处理数据
    courses_df['Effective Size'] = courses_df.apply(
        lambda x: x['Real Size'] if x['Real Size'] > 0 else x['Planned Size'], 
        axis=1
    )

    # 将教学周解析为列表
    def parse_weeks(pattern):
        if not isinstance(pattern, str):
            return []
        weeks = []
        for part in pattern.split(','):
            part = part.strip()
            if '-' in part:
                start, end = map(int, part.split('-'))
                weeks.extend(range(start, end + 1))
            else:
                try:
                    weeks.append(int(part))
                except:
                    pass
        return sorted(weeks)

    courses_df['Teaching Weeks'] = courses_df['Teaching Week Pattern'].apply(parse_weeks)

    # 创建时间槽标识
    courses_df['Time Slot'] = courses_df['Scheduled Days'] + '_' + courses_df['Scheduled Start Time']

    # 按教室容量降序排序
    rooms_sorted = rooms_df.sort_values('CAP', ascending=False)

    # 按课程大小降序排序
    courses_sorted = courses_df.sort_values('Effective Size', ascending=False)

    # 初始化结果存储
    final_schedule = []
    room_occupancy = {}  # 追踪每个教室的占用情况

    # 遍历每门课程
    for _, course in courses_sorted.iterrows():
        best_room = None
        best_utilization = 0

        # 遍历教室
        for _, room in rooms_sorted.iterrows():
            # 容量约束检查
            if room['CAP'] >= course['Effective Size']:
                # 检查时间和周次冲突
                conflict = False
                for week in course['Teaching Weeks']:
                    key = (room['ROOM NAME'], week, course['Time Slot'])
                    if key in room_occupancy:
                        conflict = True
                        break
                
                # 如果没有冲突
                if not conflict:
                    # 计算利用率
                    utilization = course['Effective Size'] / room['CAP']
                    
                    # 选择利用率最高的教室
                    if utilization > best_utilization:
                        best_room = room['ROOM NAME']
                        best_utilization = utilization

        # 如果找到合适的教室
        if best_room:
            # 为每个教学周创建排课记录
            for week in course['Teaching Weeks']:
                final_schedule.append({
                    '课程名称': course['Activity'],
                    '课程代码': course['Course Code'],
                    '活动类型': course['Activity Type Name'],
                    '教室': best_room,
                    '教学周': week,
                    '时间段': course['Time Slot'],
                    '课程人数': course['Effective Size'],
                    '教室容量': rooms_df[rooms_df['ROOM NAME'] == best_room]['CAP'].values[0],
                    '教室利用率': best_utilization,
                    '开始时间': course['Scheduled Start Time'],
                    '结束时间': course['Scheduled End Time']
                })

                # 标记教室占用
                room_occupancy[(best_room, week, course['Time Slot'])] = course['Activity']

    # 转换为DataFrame
    schedule_df = pd.DataFrame(final_schedule)

    # 结果分析
    print("教室分配结果统计：")
    print(f"总课程数：{len(schedule_df['课程名称'].unique())}")
    print(f"平均教室利用率：{schedule_df['教室利用率'].mean():.2%}")
    print(f"最高利用率：{schedule_df['教室利用率'].max():.2%}")
    print(f"最低利用率：{schedule_df['教室利用率'].min():.2%}")

    # 保存详细排课表
    schedule_df.to_excel('detailed_course_schedule.xlsx', index=False)

    return schedule_df

# 读取数据
courses_df = pd.read_excel('df_final_cleaned_1.xlsx')
rooms_df = pd.read_excel('Timetabling KB Rooms.xlsx')

# 运行优化
result = optimize_course_scheduling(courses_df, rooms_df)

# 显示部分排课表
print("\n排课表预览（前20行）：")
print(result.head(20))

教室分配结果统计：
总课程数：498
平均教室利用率：77.12%
最高利用率：100.00%
最低利用率：12.50%

排课表预览（前20行）：
                                                 课程名称       课程代码      活动类型  \
0   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
1   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
2   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
3   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
4   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
5   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
6   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
7   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
8   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
9   MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *Lecture   
10  MATH1 Engineering Mathematics/ Mathematics for...  MATH08075  *

In [ ]:
# 模型2:多目标优化，需要求解器，还没安装下来
import pandas as pd
import numpy as np
import pulp
import time
from datetime import datetime
import matplotlib.pyplot as plt

def optimize_course_scheduling():
    """
    优化排课模型，基于给定的课程和教室数据，生成最优的教室分配方案
    目标函数：最小化 f = (f1, f2)，其中：
    - f1: 时间段利用率目标 (减少冲突，提高利用率)
    - f2: 教室容量合理性目标 (教室容量应接近但不小于课程人数)
    """
    print("开始教室排课优化...")
    start_time = time.time()
    
    # 1. 读取数据
    print("1. 读取课程和教室数据...")
    courses_df = pd.read_excel('df_final_cleaned_1.xlsx')
    rooms_df = pd.read_excel('Timetabling KB Rooms.xlsx')
    
    # 2. 数据预处理
    print("2. 预处理数据...")
    
    # 清理和准备课程数据
    courses_df['Real Size'] = pd.to_numeric(courses_df['Real Size'], errors='coerce').fillna(0)
    courses_df['Planned Size'] = pd.to_numeric(courses_df['Planned Size'], errors='coerce').fillna(0)
    # 使用实际人数，若实际人数为0则使用计划人数
    courses_df['Effective Size'] = courses_df.apply(lambda x: x['Real Size'] if x['Real Size'] > 0 else x['Planned Size'], axis=1)
    
    # 清理教学周格式：将 "26-30, 32-37" 转换为 [26,27,28,29,30,32,33,34,35,36,37]
    def parse_week_pattern(pattern):
        if not isinstance(pattern, str):
            return []
        
        weeks = []
        parts = pattern.split(',')
        
        for part in parts:
            part = part.strip()
            if '-' in part:
                start, end = map(int, part.split('-'))
                weeks.extend(range(start, end + 1))
            else:
                try:
                    weeks.append(int(part))
                except:
                    pass
        
        return sorted(weeks)
    
    courses_df['Teaching Weeks'] = courses_df['Teaching Week Pattern'].apply(parse_week_pattern)
    
    # 清理时间格式
    def convert_time_to_minutes(time_str):
        if not isinstance(time_str, str):
            return 0
        try:
            hours, minutes = map(int, time_str.split(':'))
            return hours * 60 + minutes
        except:
            return 0
    
    courses_df['Start Minutes'] = courses_df['Scheduled Start Time'].apply(convert_time_to_minutes)
    courses_df['End Minutes'] = courses_df['Scheduled End Time'].apply(convert_time_to_minutes)
    
    # 创建时间段标识符 (星期+开始时间)
    courses_df['Time Slot'] = courses_df['Scheduled Days'] + '_' + courses_df['Scheduled Start Time']
    
    # 确保教室容量为数值
    rooms_df['CAP'] = pd.to_numeric(rooms_df['CAP'], errors='coerce').fillna(0)
    
    # 3. 定义问题变量
    print("3. 设置优化问题...")
    
    # 获取唯一的课程、教室、时间段和教学周
    courses = courses_df['Activity'].tolist()
    rooms = rooms_df['ROOM NAME'].tolist()
    time_slots = courses_df['Time Slot'].unique().tolist()
    all_weeks = set()
    for weeks in courses_df['Teaching Weeks']:
        all_weeks.update(weeks)
    all_weeks = sorted(list(all_weeks))
    
    # 创建课程字典，用于快速查找
    course_dict = {}
    for _, row in courses_df.iterrows():
        course_dict[row['Activity']] = {
            'size': row['Effective Size'],
            'weeks': row['Teaching Weeks'],
            'time_slot': row['Time Slot'],
            'current_room': row['Allocated Location Name'],
            'course_code': row['Course Code'],
            'activity_type': row['Activity Type Name']
        }
    
    # 创建教室字典，用于快速查找
    room_dict = {}
    for _, row in rooms_df.iterrows():
        room_dict[row['ROOM NAME']] = {
            'capacity': row['CAP'],
            'campus': row['CAMPUS'],
            'building': row['BUILDING'],
            'layout': row['SUIT 4 - ROOM LAYOUT'] if 'SUIT 4 - ROOM LAYOUT' in row else ''
        }
    
    # 4. 创建优化问题
    problem = pulp.LpProblem("Course_Room_Assignment", pulp.LpMinimize)
    
    # 创建决策变量：x[i,j,k,t] = 1表示课程i在教学周k的时间段t分配给教室j
    x = {}
    for i, course in enumerate(courses):
        for j, room in enumerate(rooms):
            for k in course_dict[course]['weeks']:
                t = course_dict[course]['time_slot']
                x[i, j, k, t] = pulp.LpVariable(f"x_{i}_{j}_{k}_{t}", cat=pulp.LpBinary)
    
    # 5. 添加约束条件
    print("4. 添加约束条件...")
    
    # 约束1：每个课程在每个教学周和时间段只能分配到一个教室
    for i, course in enumerate(courses):
        for k in course_dict[course]['weeks']:
            t = course_dict[course]['time_slot']
            problem += pulp.lpSum([x[i, j, k, t] for j in range(len(rooms))]) == 1, f"course_{i}_week_{k}_timeslot_{t}_assignment"
    
    # 约束2：同一个教室在同一个教学周和时间段不能分配给多个课程
    for j, room in enumerate(rooms):
        for k in all_weeks:
            for t in time_slots:
                courses_in_slot = [i for i, course in enumerate(courses) 
                                if k in course_dict[course]['weeks'] and course_dict[course]['time_slot'] == t]
                if courses_in_slot:
                    problem += pulp.lpSum([x[i, j, k, t] for i in courses_in_slot]) <= 1, f"room_{j}_week_{k}_timeslot_{t}_capacity"
    
    # 约束3：确保教室容量满足课程需求
    for i, course in enumerate(courses):
        for j, room in enumerate(rooms):
            for k in course_dict[course]['weeks']:
                t = course_dict[course]['time_slot']
                # 如果课程分配给该教室，则教室容量必须大于等于课程人数
                course_size = course_dict[course]['size']
                room_capacity = room_dict[rooms[j]]['capacity']
                
                # 使用大M法添加容量约束
                M = 1000  # 一个足够大的数
                problem += course_size - M * (1 - x[i, j, k, t]) <= room_capacity, f"size_req_{i}_{j}_{k}_{t}"
    
# 6. 设置目标函数
    print("5. 设置目标函数...")
    
    # 直接优化教室利用率
    room_utilization = []
    for i, course in enumerate(courses):
        for j, room in enumerate(rooms):
            for k in course_dict[course]['weeks']:
                t = course_dict[course]['time_slot']
                course_size = course_dict[course]['size']
                room_capacity = room_dict[rooms[j]]['capacity']
                
                # 计算利用率
                if room_capacity > 0:
                    # 最大化利用率 = 课程人数 / 教室容量
                    # 使用负号是因为pulp默认是最小化问题
                    room_utilization.append(-1 * (course_size / room_capacity) * x[i, j, k, t])

    # 设置目标函数为最大化利用率
    problem += pulp.lpSum(room_utilization)
    
# 7. 求解问题
    print("6. 求解优化问题...")
    
    # 尝试多个求解器
    solvers = [
        pulp.GLPK_CMD(msg=False),  # 推荐使用GLPK
        pulp.PULP_CBC_CMD(msg=False),
        pulp.COIN_CMD(msg=False)
    ]
    
    solved = False
    for solver in solvers:
        try:
            problem.solve(solver)
            
            if problem.status == pulp.LpStatusOptimal:
                solved = True
                print(f"使用 {solver} 求解成功")
                break
        except Exception as e:
            print(f"求解器 {solver} 失败: {e}")
    
    if not solved:
        print("所有求解器都失败，无法找到最优解")
        return None
    
    # 8. 分析结果
    if problem.status == pulp.LpStatusOptimal:
        print("7. 分析优化结果...")
        
        # 创建新的分配结果
        new_assignments = []
        
        for i, course in enumerate(courses):
            course_info = course_dict[course]
            
            for k in course_info['weeks']:
                t = course_info['time_slot']
                for j, room in enumerate(rooms):
                    if pulp.value(x[i, j, k, t]) == 1:
                        new_assignments.append({
                            'Activity': course,
                            'Course Code': course_info['course_code'],
                            'Activity Type': course_info['activity_type'],
                            'Size': course_info['size'],
                            'Time Slot': t,
                            'Week': k,
                            'Original Room': course_info['current_room'],
                            'New Room': room,
                            'Room Capacity': room_dict[room]['capacity'],
                            'Utilization': course_info['size'] / room_dict[room]['capacity'] if room_dict[room]['capacity'] > 0 else 0,
                            'Changed': room != course_info['current_room']
                        })
        
        # 将结果转换为DataFrame
        results_df = pd.DataFrame(new_assignments)
        
        # 9. 计算优化效果
        print("8. 计算优化效果...")
        
        # 计算优化前后的平均教室利用率
        original_utilization = courses_df['Utilization Rate'].mean()
        new_utilization = results_df['Utilization'].mean()
        
        # 计算改变的课程比例
        changed_courses = len(results_df[results_df['Changed']])
        total_assignments = len(results_df)
        change_percentage = (changed_courses / total_assignments) * 100
        
        print(f"优化前平均教室利用率: {original_utilization:.2%}")
        print(f"优化后平均教室利用率: {new_utilization:.2%}")
        print(f"利用率提升: {(new_utilization - original_utilization) * 100:.2f}个百分点")
        print(f"改变的课程分配数量: {changed_courses} / {total_assignments} ({change_percentage:.2f}%)")
        
        # 10. 保存结果
        print("9. 保存优化结果...")
        output_filename = 'optimized_course_schedule.xlsx'
        results_df.to_excel(output_filename, index=False)
        print(f"优化后的排课结果已保存至: {output_filename}")
        
        # 11. 创建可视化比较
        print("10. 创建优化结果可视化...")
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # 原始利用率分布
        original_util_bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
        original_util_labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
        original_util_counts = [0, 0, 0, 0, 0]
        
        for util in courses_df['Utilization Rate']:
            if util < 0.2:
                original_util_counts[0] += 1
            elif util < 0.4:
                original_util_counts[1] += 1
            elif util < 0.6:
                original_util_counts[2] += 1
            elif util < 0.8:
                original_util_counts[3] += 1
            else:
                original_util_counts[4] += 1
        
        # 新的利用率分布
        new_util_counts = [0, 0, 0, 0, 0]
        for util in results_df['Utilization']:
            if util < 0.2:
                new_util_counts[0] += 1
            elif util < 0.4:
                new_util_counts[1] += 1
            elif util < 0.6:
                new_util_counts[2] += 1
            elif util < 0.8:
                new_util_counts[3] += 1
            else:
                new_util_counts[4] += 1
        
        # 绘制原始利用率分布
        ax1.bar(original_util_labels, original_util_counts, color='blue', alpha=0.6)
        ax1.set_title('优化前教室利用率分布')
        ax1.set_xlabel('利用率区间')
        ax1.set_ylabel('课程数量')
        
        # 绘制新的利用率分布
        ax2.bar(original_util_labels, new_util_counts, color='green', alpha=0.6)
        ax2.set_title('优化后教室利用率分布')
        ax2.set_xlabel('利用率区间')
        ax2.set_ylabel('课程数量')
        
        plt.tight_layout()
        plt.savefig('utilization_comparison.png')
        plt.show()
        
        # 计算执行时间
        end_time = time.time()
        print(f"优化完成，总执行时间: {end_time - start_time:.2f}秒")
        
        return results_df
    else:
        print("优化未找到可行解，请调整模型参数或约束条件。")
        return None

# 运行优化模型
optimized_schedule = optimize_course_scheduling()

开始教室排课优化...
1. 读取课程和教室数据...
2. 预处理数据...
3. 设置优化问题...
4. 添加约束条件...
5. 设置目标函数...
6. 求解优化问题...
求解器 <pulp.apis.glpk_api.GLPK_CMD object at 0x174e032c0> 失败: PuLP: Error while trying to execute glpsol
求解器 <pulp.apis.coin_api.PULP_CBC_CMD object at 0x166fa74a0> 失败: PULP_CBC_CMD: Not Available (check permissions on /Users/ashley/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/arm64/cbc)
所有求解器都失败，无法找到最优解
